<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_3/03_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMLによる「異常検知」
ライブラリPyCaretを使い、AutoMLによる「異常検知」を実装します。  
様々なタンパク質の測定値から、異常なサンプルを検出します。  

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install numpy==1.21.4 numba==0.53
!pip install pycaret==2.3.10
!pip install pandas-profiling==3.1.0

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データセットの読み込み

今回は、UCIのMice Protein Expressionというデータセットを使います。  
このデータセットは、大脳皮質の核の部分で検出可能なシグナルを出した、77のタンパク質／タンパク質修飾の発現レベルで構成されています。  
このデータセットには、タンパク質ごとに合計1080の測定値が含まれおり、各測定値は独立したマウスのものです。  

https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression

In [ ]:
from pycaret.datasets import get_data

dataset = get_data("mice")  # データの取得
dataset.shape  # データの形状

訓練済みモデルの検証に用いる「未知のデータ」を訓練データから取り出します。  

In [ ]:
data = dataset.sample(frac=0.9, random_state=786)  # 訓練データ
data_unseen = dataset.drop(data.index)  # 未知のデータ

data.reset_index(drop=True, inplace=True)  # インデックスを初期化
data_unseen.reset_index(drop=True, inplace=True)  # インデックスを初期化

print("訓練データの形状: " + str(data.shape))
print("未知のデータの形状: " + str(data_unseen.shape))

## 環境の設定
PyCaretの環境を設定します。  
setup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。  

以下のコードを実行すると、setup関数が実行されてすべての特徴のデータ型が自動的に推定されます。  
この際に、必ずしも正しく推論されるとは限らないことにご注意ください。     
データの型に問題が無ければ、空白を入力することで設定を完了することができます。  

In [ ]:
from pycaret.anomaly import setup

clf = setup(data=data, normalize = True, ignore_features = ['MouseID'], session_id=123)  # 環境の初期化

## モデルの作成
models関数により、全ての使用可能な異常検知のための機械学習モデルを確認することができます。

In [ ]:
from pycaret.anomaly import models

models()  # 機械学習モデルの一覧

今回は、異常検知でよく使われるOne Class Support Vector Machine（OCSVM、One Class SVM）を使用します。  
One Class SVMは大多数が正常であるようなデータセットをもとに学習を行い、未知のデータが正常であるのか、異常であるのかを判定するのに適しています。   
以下のコードではcreate_modelで"svm"を設定し、One Class SVMを選択しています。  
また、fractionで異常の割合を指定しています。


In [ ]:
from pycaret.anomaly import create_model

svm = create_model("svm", fraction=0.025)

訓練済みモデルの概要を表示します。  

In [ ]:
print(svm)

One class SVMは、サポートベクターマシンの応用であるにも関わらず教師なし学習となります。  
One class SVMでは、学習データを1つのクラスタ とし、原点をもう一つのクラスタとするカーネルトリックという手法を使います。   
これにより高次元空間にデータが写像されるのですが、学習データは原点から遠くに配置されるため、学習データから離れたサンプルは原点の近くに集まります。  
これにより、正常/異常の区別をします。  

## ラベルの割当て
`assign_model`関数により、サンプルが異常かどうかを表すラベル、および異常の程度を表すスコアを割り当てることができます。  

In [ ]:
from pycaret.anomaly import assign_model

kmean_assigned = assign_model(svm)
kmean_assigned.head()

OCSVMのAnomaly Scoreの計算方法ですが、SVMの超平面からの「符号付き距離 」がAnomaly Scoreになります。  
符号付距離は、正常なサンプルの場合は正、異常な場合は負となります。  
詳しくは、以下のscikit-learnの公式ドキュメントが参考になります。  
https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html#sklearn.svm.OneClassSVM.decision_function


## モデルを評価する
plot_model関数により、様々な角度からモデルを分析することができます。  
https://pycaret.org/plot-model/

plot_model関数でplotを指定しない場合、t-SNEの結果が表示されます。  

In [ ]:
from pycaret.anomaly import plot_model

plot_model(svm)

## モデルの保存と読み込み

`save_model`関数によりモデルを保存することができます。  

In [ ]:
from pycaret.anomaly import save_model

save_model(svm, "model_anomaly_detection")

`load_model`関数により、保存されたモデルを読み込むことができます。

In [ ]:
from pycaret.anomaly import load_model

loaded_svm = load_model("model_anomaly_detection")

## 未知のデータで予測する
「未知のデータ」が所属すべきクラスタを予測します。  

In [ ]:
from pycaret.anomaly import predict_model

unseen_predictions = predict_model(loaded_svm, data=data_unseen)
unseen_predictions.head()